In [79]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
import time
import warnings
import numpy as np


In [58]:
driver = webdriver.Chrome()
driver.get("http://books.toscrape.com/")

In [57]:
def book_scraping():
    
    driver = webdriver.Chrome()
    driver.get("http://books.toscrape.com/")
    book_list = pd.DataFrame(columns = ['Title','Price','Rating','Available'])

    n_pages = (int(driver.find_element_by_class_name('current').text[-2:]))

    for i in range(n_pages):
        driver.implicitly_wait(4)
        soup = bs(driver.page_source, parser="lxml")
        for book in soup.find("ol", class_="row").find_all("li"):

            book_name = book.find("article", class_="product_pod").find("h3").a["title"]
            book_price = (book.find("article", class_="product_pod").find("div", class_="product_price").p.text)
            book_rating = book.find("article", class_="product_pod").p["class"][1]
            book_available = (book.find("article", class_="product_pod")
                              .find("div", class_="product_price")
                              .find("p", class_="instock availability")
                              .text.lstrip("\n\n    \n        ").rstrip("\n    \n"))

            book_list = book_list.append({'Title':book_name, 'Price':book_price, 'Rating':book_rating, 'Available':book_available}, ignore_index = True)
        print('Scraped page ', i+1)

        if i < n_pages-1:
            driver.find_element_by_class_name('next').find_element_by_tag_name('a').click()
            
    return book_list

In [159]:
book_list.append(book_list)

,Title,Price,Rating,Available,Genre
0,A Light in the Attic,£51.77,Three,In stock,Poetry
1,Tipping the Velvet,£53.74,One,In stock,Historical Fiction
2,Soumission,£50.10,One,In stock,Fiction
3,Sharp Objects,£47.82,Four,In stock,Mystery
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock,History
...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,One,In stock,Classics
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,Four,In stock,Other
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,Five,In stock,Other
998,1st to Die (Women's Murder Club #1),£53.98,One,In stock,Other


In [151]:
def genre_classification(book_list):
    
    book_list['Genre'] = 'Other'
    
    for cat in range(len(driver.find_element_by_class_name('side_categories').find_element_by_tag_name('ul').find_element_by_tag_name('ul').find_elements_by_tag_name('li'))):
        cur_cat = driver.find_element_by_class_name('side_categories').find_element_by_tag_name('ul').find_element_by_tag_name('ul').find_elements_by_tag_name('li')[cat]
        category = cur_cat.find_element_by_tag_name('a').text

        cur_cat.find_element_by_tag_name('a').click()


        soup = bs(driver.page_source, 'lxml')

        for book in soup.find("ol", class_="row").find_all("li"):

            book_name = book.find("article", class_="product_pod").find("h3").a["title"]
            book_list.loc[book_list.Title == book_name, 'Genre'] = category

    return book_list

In [149]:
book_list[15:45]

,Title,Price,Rating,Available,Genre
15,Our Band Could Be Your Life: Scenes from the A...,£57.25,Three,In stock,Music
16,Olio,£23.88,One,In stock,Poetry
17,Mesaerion: The Best Science Fiction Stories 18...,£37.59,One,In stock,Science Fiction
18,Libertarianism for Beginners,£51.33,Two,In stock,Politics
19,It's Only the Himalayas,£45.17,Two,In stock,Travel
20,In Her Wake,£12.84,One,In stock,Thriller
21,How Music Works,£37.32,Two,In stock,Music
22,Foolproof Preserving: A Guide to Small Batch J...,£30.52,Three,In stock,Food and Drink
23,Chase Me (Paris Nights #2),£25.27,Five,In stock,Romance
24,Black Dust,£34.53,Five,In stock,Romance
